# Extracting Features from Textual data in Spark

## Initializing Spark
We initialize spark first and configure the number of executors, cores, and executor memory. Spark does not have a built-in function to do the stemming. So we will use a third party package from here: https://github.com/master/spark-stemming to do the stemming. We have to add this package to our spark configuration by setting launcher.packages parameter.

In [1]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_cores=2
launcher.executor_memory='2600m'
launcher.packages=["com.github.master:spark-stemming_2.10:0.2.0"]

## Loading the reviews from Yelp
We are going to use the Yelp review file containing the text of millions of user reviews on Yelp. The file is downloaded from here (https://www.yelp.com/dataset/challenge). We have downloaded the dataset and copied the review.json file on hdfs.  The following line loads the data in spark, and create a data frme with only "Text" column containing review texts. We cache the data frame (i.e., persist it in memory because we will reuse it in subsequent configurations)

In [4]:
val reviewsDF=spark.read.json("/hadoop-user/data/yelp/review.json").select("Text")
reviewsDF.show(3)
reviewsDF.cache()

+--------------------+
|                Text|
+--------------------+
|Love the staff, l...|
|Super simple plac...|
|Small unassuming ...|
+--------------------+
only showing top 3 rows

2018-08-23 12:24:37 WARN  CacheManager:66 - Asked to cache already cached data.


reviewsDF: org.apache.spark.sql.DataFrame = [Text: string]
res2: reviewsDF.type = [Text: string]


## Tokenizing
The first thing we need to do is to convert each review text to an array of its words (i.e., tokenize  it). The "RegexTokenizer" in spark  is a transformer which tokenize a column of text and convert each text to a sequence of its words. I have set the parameter "setMinTokenLength" to 2 to remove any words that is less than two characters. I have also coverted all words to lower case by setting the setToLowerCase parameter to true.

In addition, I would like to remove all the punctuations from each word. So I defined a udf removePunc which takes a sequence of words and remove all punctuations in each word in the sequence. I apply this udf to the column returned by the Tokenizer.

In [11]:
import org.apache.spark.ml.feature._
val tokenized = new RegexTokenizer().setMinTokenLength(2).setToLowercase(true).setInputCol("Text").setOutputCol("words").
transform(reviewsDF)

println("A sample of the output of the tokenized reviews before removing punctuations")
tokenized.show(5, truncate=55)

//Defining a udf to remove punctuations from a sequence of words
import org.apache.spark.sql.functions.udf

def removePunc(words:Seq[String]):Seq[String]={
 return words.map(_.replaceAll("\\p{Punct}",""))
}

val removePuncUDF=udf(removePunc(_:Seq[String]))

//use the removePuncUDF to remove all punctuations from words
val reviewWords= tokenized.withColumn("words",removePuncUDF($"words"))

println("A sample of the tokenized reviews after removing punctuations")
reviewWords.show(5, truncate=55)




A sample of the output of the tokenized reviews before removing punctuations
+-------------------------------------------------------+-------------------------------------------------------+
|                                                   Text|                                                  words|
+-------------------------------------------------------+-------------------------------------------------------+
|Love the staff, love the meat, love the place. Prepa...|[love, the, staff,, love, the, meat,, love, the, pla...|
|Super simple place but amazing nonetheless. It's bee...|[super, simple, place, but, amazing, nonetheless., i...|
|Small unassuming place that changes their menu every...|[small, unassuming, place, that, changes, their, men...|
|Lester's is located in a beautiful neighborhood and ...|[lester's, is, located, in, beautiful, neighborhood,...|
|Love coming here. Yes the place always needs the flo...|[love, coming, here., yes, the, place, always, needs...|
+----------

import org.apache.spark.ml.feature._
tokenized: org.apache.spark.sql.DataFrame = [Text: string, words: array<string>]
import org.apache.spark.sql.functions.udf
removePunc: (words: Seq[String])Seq[String]
removePuncUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(StringType,true),Some(List(ArrayType(StringType,true))))
reviewWords: org.apache.spark.sql.DataFrame = [Text: string, words: array<string>]


## Removing Stop Words and Stemming
After tokenizing and removing the punctuations, we should remove the stop words (The words that occur frequently in text but do not carry much information e.g., "the", "I", "of", etc.).
We can use spark's built-in transformer StopWordRemover to remove stop words. Unfortunately, spark does not have a built-in transformer to do stemming so we'll use a third party package to do so (https://github.com/master/spark-stemming)

In [25]:
//removing stop words using spark's StopWordRemover transformer
val reviews_filtered=new StopWordsRemover().setInputCol("words").setOutputCol("filtered_words").transform(reviewWords).select("Text","filtered_words")
println("reviews after removing stop words")
reviews_filtered.show(2, truncate=55)


//Stemming using a third party package
import org.apache.spark.mllib.feature.Stemmer
val stemmed_reviews = new Stemmer().setInputCol("filtered_words").setOutputCol("stemmed_words").transform(reviews_filtered)
println("After stemming")
stemmed_reviews.select("Text","stemmed_words").show(2, truncate=60)


reviews after removing stop words
+-------------------------------------------------------+-------------------------------------------------------+
|                                                   Text|                                         filtered_words|
+-------------------------------------------------------+-------------------------------------------------------+
|Love the staff, love the meat, love the place. Prepa...|[love, staff, love, meat, love, place, prepare, long...|
|Super simple place but amazing nonetheless. It's bee...|[super, simple, place, amazing, nonetheless, around,...|
+-------------------------------------------------------+-------------------------------------------------------+
only showing top 2 rows

After stemming
+------------------------------------------------------------+------------------------------------------------------------+
|                                                        Text|                                               stemmed_w

reviews_filtered: org.apache.spark.sql.DataFrame = [Text: string, filtered_words: array<string>]
import org.apache.spark.mllib.feature.Stemmer
stemmed_reviews: org.apache.spark.sql.DataFrame = [Text: string, filtered_words: array<string> ... 1 more field]


## Vectorizing words using CountVectorizer
CountVectorizer is a spark estimator which takes a column of word vectors and generates a BAg of Word (BOW) for each word vector (please refer to the lectures for more details on bag of words). The BOW generated for each word vector is a sparse vector in form of (size,[indices],[values]) where "size" is the size of the vocabulary (i.e., total number of distinct words in the column), indices, are the indices of the words that occur in the word vector, and values are their frequencies in the word vector. For example, if we have a dataframe with columns id and words:

 id | words
----|----------
 0  | Array("a", "b", "c","d")
 1  | Array("a", "b", "b", "c", "a")
 
 Then fitting the countVectorizer will produce a CountVectorizerModel with vocabulary (a,b,c,d) and a BOW for each word vector as follows:
 
  id | texts                           | BOW
----|---------------------------------|---------------
 0  | Array("a", "b", "c","d","a")            | (4,[0,1,2,3],[2.0,1.0,1.0,1.0])
 1  | Array("a", "b", "b", "d", "a")  | (4,[0,1,3],[2.0,2.0,1.0])
 
The BOW for the word vector Array("a", "b", "b", "c", "a") means that the vocabulary size is 4 and this word vector has three words with indices (0="a", 1="b","d"=3) where "a" and "b" occur twice and "d" occurs only once. Note that "c" is at index 2 and its frequency in this word vector is zero.

Now let's apply CountVectorizer to our yelp reviews. The CountVectorizer has an optional parameter "MinDF" which shows the fminimum fraction of Documents/rows (or in this case reviews) that a word should appear to be included in the vocabulary.We will set this to 0.05 meaning that  word should occur at least in 5% of the reviews to be considered in the vocabulary.
Be patient when you run this code segment. It makes several passes through data and may take a few minutes to complete.

In [35]:
val bow = new CountVectorizer().setInputCol("stemmed_words").setOutputCol("BOW").setMinDF(0.05).fit(stemmed_reviews).transform(stemmed_reviews)
bow.show(3)
reviewsDF.unpersist()
bow.cache()

+--------------------+--------------------+--------------------+--------------------+
|                Text|      filtered_words|       stemmed_words|                 BOW|
+--------------------+--------------------+--------------------+--------------------+
|Love the staff, l...|[love, staff, lov...|[love, staff, lov...|(185,[0,5,15,23,2...|
|Super simple plac...|[super, simple, p...|[super, simpl, pl...|(185,[0,13,36,55,...|
|Small unassuming ...|[small, unassumin...|[small, unassum, ...|(185,[0,7,12,26,2...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



bow: org.apache.spark.sql.DataFrame = [Text: string, filtered_words: array<string> ... 2 more fields]
res32: bow.type = [Text: string, filtered_words: array<string> ... 2 more fields]


## Generating TFIDF vectors
Term-Frequency-Inverse-Document-Frequency (TFIDF) are BOW vectors normalized by the number of documents in which the word occurs (please refer to the lectures for detailed formula). You can use the IDF estimator in spark to compute TFIDF. IDF takes a column of BOW as input and computers the TFIDF for each word in BOW. 

The TFIDF features generated this way can be passed to a learning algorithm.

Let's get the TFIDF vectors for our yelp reviews. The following code segment may take several minutes to run so be patient.

In [42]:
val tfidf = new IDF().setInputCol("BOW").setOutputCol("TFIDF features").fit(bow).transform(bow)
println("A sample of TFIDF feature vectors")
tfidf.select("TFIDF features").show(5,truncate=70)

A sample of TFIDF feature vectors
+----------------------------------------------------------------------+
|                                                        TFIDF features|
+----------------------------------------------------------------------+
|(185,[0,5,15,23,25,36,38,54,69,84,92,127,131,132,135,138,156],[0.98...|
|(185,[0,13,36,55,56,82,90,92,105,130,146,150],[0.9847781203458608,1...|
|(185,[0,7,12,26,27,46,49,56,59,62,81,89,100,111,112,114,118,125,129...|
|(185,[7,16,22,28,58,66,99,105,129,161],[1.2652801896236774,1.702332...|
|(185,[0,1,2,5,10,15,16,34,46,60,63,67,69,76,118,125,178],[0.9847781...|
+----------------------------------------------------------------------+
only showing top 5 rows



tfidf: org.apache.spark.sql.DataFrame = [Text: string, filtered_words: array<string> ... 3 more fields]
